# This script contains the following points:

#### 01. Import libraries

#### 02. Import data set - customer

#### 03. Data wrangling

#### 04. Fundamental data quality and consistency checks

#### 05. Combine data set - customer with orders_products_merged_2_updated


# 01. Import libraries


In [30]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

# 02. Import data set

In [31]:
# Set path

path = r'C:\Users\nang6\OneDrive\Bureau\Data Analytics\Data Immersion\Achievement 4\02-2020 Instacart Basket Analysis'

In [32]:
# Import customer

df_customer = pd.read_csv(os.path.join(path, '02 Data', 'Original Data','customers.csv'))

In [33]:
# Import orders_products_merged_2_updated

df_ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_2_updated.pkl'))

# 03. Data wrangling

In [34]:
# Check the first 5 rows

df_customer.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [35]:
# Check the last 5 rows

df_customer.tail()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
206204,168073,Lisa,Case,Female,North Carolina,44,4/1/2020,1,married,148828
206205,49635,Jeremy,Robbins,Male,Hawaii,62,4/1/2020,3,married,168639
206206,135902,Doris,Richmond,Female,Missouri,66,4/1/2020,2,married,53374
206207,81095,Rose,Rollins,Female,California,27,4/1/2020,1,married,99799
206208,80148,Cynthia,Noble,Female,New York,55,4/1/2020,1,married,57095


In [36]:
# Check size

df_customer.shape

(206209, 10)

In [37]:
# Check data types

df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


In [38]:
# Change data types for some columns in df_customer 

df_customer['user_id']=df_customer['user_id'].astype('str')
df_customer['Age']=df_customer['Age'].astype('int8')
df_customer['n_dependants']=df_customer['n_dependants'].astype('int16')
df_customer['income']=df_customer['income'].astype('int32')

In [39]:
# Drop First Name and Surnam columns as irrelevant for analysis

df_customer = df_customer.drop(columns = ['First Name', 'Surnam'])

In [40]:
# Renaming columns; Gender,STATE, Age, n_dependants and fam_status for data consistency

df_customer.rename(columns = {'Gender' : 'gender', 'STATE' : 'state', 'Age' : 'age', 'n_dependants' : 'number_dependants', 'fam_status' : 'family_status' }, inplace = True)

In [41]:
# Check outcome

df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   user_id            206209 non-null  object
 1   gender             206209 non-null  object
 2   state              206209 non-null  object
 3   age                206209 non-null  int8  
 4   date_joined        206209 non-null  object
 5   number_dependants  206209 non-null  int16 
 6   family_status      206209 non-null  object
 7   income             206209 non-null  int32 
dtypes: int16(1), int32(1), int8(1), object(5)
memory usage: 9.2+ MB


In [42]:
df_customer

,user_id,gender,state,age,date_joined,number_dependants,family_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374
...,...,...,...,...,...,...,...,...
206204,168073,Female,North Carolina,44,4/1/2020,1,married,148828
206205,49635,Male,Hawaii,62,4/1/2020,3,married,168639
206206,135902,Female,Missouri,66,4/1/2020,2,married,53374
206207,81095,Female,California,27,4/1/2020,1,married,99799


# 04. Fundamental data quality and consistency checks

In [43]:
# Check for mixed-types

for col in df_customer.columns.tolist():
  weird = (df_customer[[col]].applymap(type) != df_customer[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customer[weird]) > 0:
    print (col)

## Observation: No mixed-types found

In [44]:
# Check for missing values

df_customer.isnull().sum()

user_id              0
gender               0
state                0
age                  0
date_joined          0
number_dependants    0
family_status        0
income               0
dtype: int64

## Observation: No missing values found

In [45]:
# Check for duplicates

df_customer_dups = df_customer[df_customer.duplicated()]

In [46]:
df_customer_dups

,user_id,gender,state,age,date_joined,number_dependants,family_status,income


## Observation: No duplicates found

In [47]:
# Descriptive Statistics

df_customer.describe()

,age,number_dependants,income
count,206209.000000,206209.000000,206209.000000
mean,49.501646,1.499823,94632.852548
std,18.480962,1.118433,42473.786988
min,18.000000,0.000000,25903.000000
25%,33.000000,0.000000,59874.000000
50%,49.000000,1.000000,93547.000000
75%,66.000000,3.000000,124244.000000
max,81.000000,3.000000,593901.000000


# 05. Combine data set - customer with orders_products_merged_2_updated

In [48]:
# Check for a common column

df_ords_prods_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 24 columns):
 #   Column                         Dtype   
---  ------                         -----   
 0   order_id                       int32   
 1   user_id                        int32   
 2   order_number                   int8    
 3   orders_day_of_week             int8    
 4   order_hour_of_day              int8    
 5   days_since_prior_order         float16 
 6   product_id                     int32   
 7   add_to_cart_order              int32   
 8   reordered                      int8    
 9   product_name                   object  
 10  aisle_id                       int8    
 11  department_id                  int8    
 12  prices                         float16 
 13  _merge                         category
 14  price_range_loc                object  
 15  busiest_day                    object  
 16  busiest_days                   object  
 17  busiest_period_of_day    

## Observation: commun column is user_id

In [49]:
# Change data type for column 'user_id' in df_ords_prods_merge

df_ords_prods_merge['user_id']=df_ords_prods_merge['user_id'].astype('str')

In [50]:
# Check outcome

df_ords_prods_merge['user_id'].dtype

dtype('O')

In [51]:
df_customer['user_id'].dtype

dtype('O')

In [52]:
# Merge the two dataframes

df_merged = pd.merge(df_ords_prods_merge, df_customer, on = 'user_id', how = 'outer')

In [53]:
# check first 5 rows

df_merged.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,spending_flag,median_days_since_prior_order,order_frequency_flag,gender,state,age,date_joined,number_dependants,family_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Low spender,20.5,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Low spender,20.5,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Low spender,20.5,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Low spender,20.5,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Low spender,20.5,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423


In [54]:
# Check size

df_merged.shape

(32404859, 31)

In [55]:
#Check value counts

df_merged['_merge'].value_counts()

both          32404859
left_only            0
right_only           0
Name: _merge, dtype: int64

In [56]:
# Export data to pkl

df_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_all.pkl'))